# Retrieval augmented chat - Llama-2-7B-Chat

This notebook is the primary demonstration of the project with the baseline model. Here we'll bring up the baseline model and vector database and start asking questions both with and without the vector database.

In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

## Initialize some variables

In [2]:
collection_name = "Corpus"
model_dir = "meta-llama/Llama-2-7b-chat-hf"
device_map = {"": 0}
device = "cuda"
database_top_n_results = 2

## Load shared code
This file defines the `ChatModel` and `Retrieval` classes used below.

In [3]:
%run shared_code.ipynb

## Load the model

In [4]:
language_model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

tokenizer = AutoTokenizer.from_pretrained(model_dir)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Load the model into the ChatModel class from `shared_code.ipynb`

In [5]:
chat_model = ChatModel(language_model, tokenizer)

## Trying the model without access to the vector database.

In [6]:
chat_model.basic_chat("How do I loop a GIF?")

 There are a few ways to loop a GIF, depending on the context and the tool or software you are using. Here are a few common methods:

1. Online GIF editors: Many online GIF editors, such as Giphy or Tenor, allow you to loop a GIF by simply clicking on the "Loop" or "Repeat" button. This will repeat the GIF indefinitely.
2. Image editing software: If you are using image editing software like Adobe Photoshop or GIMP, you can use the "Animate" or "Loop" function to loop a GIF. This function allows you to create a looped animation from a series of images, including GIFs.
3. Web browsers: Some web browsers, such as Google Chrome or Mozilla Firefox, allow you to loop a GIF by right-clicking on the GIF and selecting "Loop" or "Repeat".
4. Code: If you want to loop a GIF on a website or blog, you can use HTML and JavaScript code to create a loop. Here's an example of how to do this:
```
<img src="path/to/your/gif" alt="Your GIF" loop="infinite">
```
This will loop the GIF indefinitely. You can

## Load the collection into the RetrievalAugmentedChat class from `shared_code.ipynb`

In [7]:
rac = RetrievalAugmentedChat("db/", collection_name, database_top_n_results, chat_model)

## Run retrieval augmented chat
Notice that the responses have switched from a general HTML/JavaScript context to include the document contents which provide examples of encoding the gif with ImageMagick so that it loops by default.

In [8]:
rac.markdown_chat("How do I loop a GIF?")

 To loop a GIF, you can use the `-loop` option in the ImageMagick command. The syntax for this option is as follows:
```
mogrify -loop <num> input.gif
```
Where `<num>` is the number of times you want the GIF to loop. For example, to loop a GIF 5 times, you would use the command:
```
mogrify -loop 5 input.gif
```
Alternatively, you can use the `-loop count` option, which allows you to specify the number of loops in a more concise way:
```
mogrify -loop count input.gif
```
It's important to note that the `-loop` option only works on animated GIFs, as it relies on the GIF's timeline information to loop the animation. If the GIF is not animated, the `-loop` option will have no effect.

In your case, you want to loop a GIF that you accidentally made without the `-loop` option. You can use the `mogrify` command with the `-loop` option to loop the GIF:
```
mogrify -loop 0 chrome-samesite-missing.gif
```
This will create a new GIF file called `chrome-samesite-missing-loop.gif` that loops the original animation indefinitely.

Another option for looping a GIF is to use the `convert` command with the `-loop` option:
```
convert chrome-samesite-missing.gif -loop 0 chrome-samesite-missing-loop.gif
```
This will also create a new GIF file that loops the original animation indefinitely.

In your previous question, you mentioned that you installed ImageMagick on macOS using `brew install imagemagick`. To use the `mogrify` command, you can simply open a terminal and run the command:
```
mogrify -loop 0 input.gif
```
Where `input.gif` is the name of the GIF file you want to loop.

You can also use the `convert` command to loop a GIF, like this:
```
convert input.gif -loop 0 output.gif
```
Again, where `input.gif` is the name of the GIF file you want to loop, and `output.gif` is the name of the new, looped GIF file.

I hope this helps! Let me know if you have any other questions.

 **Reference documents:** 

* [corpus/imagemagick/set-a-gif-to-loop.md](corpus/imagemagick/set-a-gif-to-loop.md) distance: 0.97
* [corpus/imagemagick/compress-animated-gif.md](corpus/imagemagick/compress-animated-gif.md) distance: 1.28

**Inference time in seconds 11.3920**


In [9]:
rac.markdown_chat("Can I use npx with GitHub actions?")

 Yes, you can use `npx` with GitHub Actions. In fact, you can use any command-line tool or utility that you would normally use in a Node.js project, including `npx`.

To use `npx` with GitHub Actions, you can leverage the `actions/cache` mechanism, which allows you to cache dependencies and other assets between runs of your workflow.

Here's an example of how you can use `npx` in a GitHub Actions workflow:

1. In your workflow file, use the `uses` keyword to install the `actions/cache` action:
```yaml
uses: actions/cache@v2
```
2. In your workflow, use the `cache` keyword to specify the cache key. The cache key should be a unique identifier for your workflow, and it can be a file path, a directory path, or a URL. For example:
```yaml
cache:
  - keys:
    - ${{ steps.cache-npx.outputs.cache-key }}
    - ${{ env.GITHUB_ACTION_ROLE }}
```
In this example, the cache key is the output of the `cache-npx` step, which is a file path.

3. Use the `npx` command in your workflow, just like you would in a Node.js project:
```yaml
  - name: Install dependencies
    run: npx install
  - name: Run a script
    run: npx run-script.js
```
The `npx` command will be executed in the context of the cache, so it will use the cached dependencies instead of downloading them again.

4. To invalidate the cache for a specific step, you can use the `invalidateCache` keyword. For example:
```yaml
  - name: Install dependencies
    run: npx install
    invalidateCache: true
```
This will invalidate the cache for the `install` step, so the next time the workflow runs, the `npx install` command will download the dependencies again.

5. To cache the output of a step, you can use the `cache` keyword with the `outputs` keyword. For example:
```yaml
  - name: Run a script
    run: npx run-script.js
    outputs:
      - cache-key
    cache: ${{ steps.cache-npx.outputs.cache-key }}
```
This will cache the output of the `run-script.js` step, so the next time the workflow runs, the `npx run-script.js` command will use the cached output instead of running the script again.

In summary, you can use `npx` with GitHub Actions by leveraging the `actions/cache` mechanism and specifying the cache key appropriately. This allows you to cache dependencies and other assets between runs of your workflow, which can improve the performance and efficiency of your workflows.

 **Reference documents:** 

* [corpus/github-actions/npm-cache-with-npx-no-package.md](corpus/github-actions/npm-cache-with-npx-no-package.md) distance: 0.88
* [corpus/github-actions/attach-generated-file-to-release.md](corpus/github-actions/attach-generated-file-to-release.md) distance: 1.11

**Inference time in seconds 13.0496**
